# 📊 AWS Glue & PySpark ETL: Análise de Transações Financeiras

# 📌 1️⃣ Introdução e Objetivo

Este notebook implementa a etapa de **ETL (Extração, Transformação e Carga)** para processamento dos dados brutos.

📌 **Objetivo:**  
- Carregar os dados brutos da pasta `data/raw/`
- Aplicar transformações com **PySpark** (limpeza, normalização e regras de negócio)
- Armazenar os dados processados em formato **Parquet** para análise posterior

🔧 **Tecnologias utilizadas:**  
- **PySpark** para processamento eficiente de grandes volumes de dados
- **AWS Glue** (se rodarmos na nuvem) ou Spark local
- **Parquet** como formato de armazenamento otimizado

---


## 📌 2️⃣ Configuração do do PySpark

In [2]:
import os
import pyspark
from pyspark.sql import SparkSession

# Criar uma sessão do PySpark
spark = SparkSession.builder \
    .appName("ETL_Processing") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

print("✅ PySpark configurado com sucesso!")

ModuleNotFoundError: No module named 'pyspark'

## 📌 3️⃣ Carregamento dos Dados Brutos

In [ ]:
# Definir caminho dos dados brutos
raw_data_path = "data/raw/"

# Listar arquivos na pasta raw
arquivos_raw = os.listdir(raw_data_path)
print(f"📂 Arquivos encontrados: {arquivos_raw}")

# Carregar os dados no PySpark DataFrame
df_raw = spark.read.csv(os.path.join(raw_data_path, "transactions.csv"), header=True, inferSchema=True)

# Exibir amostra dos dados
df_raw.show(5)

# Exibir schema dos dados
df_raw.printSchema()
